In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from nltk import word_tokenize
from collections import Counter 
import nltk

C:\Users\nr17754\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
tag_list = train['tags'].tolist()
li_flat = [] 
for i in tag_list:
    li_flat.append(str(i).split("|"))

In [4]:
flat_tags = [i for sublist in li_flat for i in sublist]
top_tags = Counter(flat_tags).most_common(500)
top_flat = []
for i in top_tags:
    top_flat.append(i[0])

In [5]:
train.shape

(786979, 4)

In [6]:
test.shape

(194323, 3)

In [7]:
## Pre-Processing Script 

# Function to remove html tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# Function to remove URLs starting with https/http 
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

# POS TAG removal
def pos_remover(x):
    tokens = word_tokenize(x)
    tagged = nltk.pos_tag(tokens)
    pos_removed_sentence = ''
    for i in tagged:
        if ((i[1]=='IN')|(i[1]=='VBZ')|(i[1]=='WP')|(i[1]=='DT')|(i[1]=='TO')|(i[1]=='PRP')|(i[1]=='VB')|(i[1]=='VBG')|(i[1]=='CC')|(i[1]=='VB')|(i[1]=='MD')):
            continue
        else:
            pos_removed_sentence = pos_removed_sentence +" "+i[0]
    return pos_removed_sentence 

# Create a unique list without repetition of words 
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist


In [8]:
def preprocessing(x):
    x = remove_urls(x)
    x = cleanhtml(x)
    x = pos_remover(x)
    x = str(x).lower()
    x = remove_stopwords(x)
    x = strip_punctuation(x)
    x = strip_multiple_whitespaces(x)
    x = ' '.join([re.sub(r'\d+', '',i) for i in word_tokenize(x)])
    x = ' '.join(unique_list(x.split()))
    return x

In [10]:
train['article'][2]

'<p>I\'m planning to use a binary system to handle my permissions, like this:</p>\r\r\r\r\n\r\r\r\r\n<pre><code>&lt;?php\r\r\r\r\nconst POST = 1;\r\r\r\r\nconst DELETE = 2;\r\r\r\r\nconst UPDATE = 4;\r\r\r\r\nconst READ = 8;\r\r\r\r\nconst ADMIN = 16;\r\r\r\r\nconst ADD_PRODUCT = 32;\r\r\r\r\nconst DELETE_PRODUCT = 64;\r\r\r\r\nconst ADD_BLOG = 128;\r\r\r\r\n\r\r\r\r\n\r\r\r\r\n$permission = decbin(\'5\');\r\r\r\r\n\r\r\r\r\nif ($permission &amp; POST) { echo "POST &lt;br /&gt;"; }\r\r\r\r\nif ($permission &amp; DELETE) { echo "DELETE &lt;br /&gt;"; }\r\r\r\r\nif ($permission &amp; UPDATE) { echo "UPDATE &lt;br /&gt;"; }\r\r\r\r\nif ($permission &amp; READ) { echo "READ &lt;br /&gt;"; }\r\r\r\r\nif ($permission &amp; ADMIN) { echo "ADMIN &lt;br /&gt;"; }\r\r\r\r\n\r\r\r\r\n?&gt;\r\r\r\r\n</code></pre>\r\r\r\r\n\r\r\r\r\n<p>With a few permissions, this will work well. But what if i have, lets say, 40 different permissions? Then I need to add up some really big numbers, which is a pain i

In [9]:
preprocessing(train['article'][2])

'm binary permissions lt php const post delete update read admin add product blog permission decbin amp echo gt lets different need big numbers pain ass way info help greatly appriciated'